<h1 style="text-align: center">
Machine Learning HW1 </br>
MLE & MAP in Python
</h1>

#### Name: Ehasan Hassanbeygi

#### Std. Number: 402211723

## Objective
This exercise will help you gain a deeper understanding of, and insights into, Maximum Likelihood Estimation (MLE) and Maximum A Posteriori (MAP) estimation$\textit{Maximum Likelihood Estimation (MLE) and Maximum A Posteriori (MAP) }$ :) \\
Let’s say you have a barrel of apples that are all different sizes. You pick an apple at random, and you want to know its weight. Unfortunately, all you have is a broken scale. answer the questions below.

1) For the sake of this section, lets imagine a farmer tells you that the scale returns the weight of the object with an error of +/- a standard deviation of 5g. We can describe this mathematically as:
$$
measurement = weight + \mathcal{N}(0, 5g)
$$
You can weigh the apple as many times as you want, so weigh it 100 times.
plot its histogram of your 100 measurements. (y axis is the counts and x-axis is the measured weight)

In [1]:
        ######################################################
        ###################### TO DO #########################
        ######################################################
        #                                                    #
        #                                                    #
        ######################################################
        ######################################################
        ######################################################

2) Find the average weight of the apple.
Is it a good guess? state your reason.

In [2]:
        ######################################################
        ###################### TO DO #########################
        ######################################################
        #                                                    #
        #                                                    #
        ######################################################
        ######################################################
        ######################################################

3) we are going to use grid approximation for calculating the MLE. here is the link if you wnat to get more fimilar with this technique:
https://www.bayesrulesbook.com/chapter-6

Our end goal is to find the weight of the apple, given the data we have. To formulate it in a Bayesian way: We’ll ask what is the probability of the apple having weight, $w$, given the measurements we took, $X$. And, because we're formulating this in a Bayesian way, we use Bayes’ Law to find the answer:

$$
P(w|X) = \frac{P(X|w)P(w)}{P(X)}
$$

If we make no assumptions about the initial weight of our apple, then we can drop $P(w)$. We’ll say all sizes of apples are equally likely (we’ll revisit this assumption in the MAP approximation).

Furthermore, we’ll drop $P(X)$ - the probability of seeing our data. This is a normalization constant and will be important if we do want to know the probabilities of apple weights. But, for right now, our end goal is to only to find the most probable weight. $P(X)$ is independent of $w$, so we can drop it if we’re doing relative comparisons.

This leaves us with $P(X|w)$, our likelihood, as in, what is the likelihood that we would see the data, $X$, given an apple of weight $w$. If we maximize this, we maximize the probability that we will guess the right weight.

The grid approximation is probably the simplest way to do this. Basically, we’ll systematically step through different weight guesses, and compare what it would look like if this hypothetical weight were to generate data. We’ll compare this hypothetical data to our real data and pick the one that matches the best.

To formulate this mathematically:

For each of these guesses, we’re asking "what is the probability that the data we have, came from the distribution that our weight guess would generate". Because each measurement is independent from another, we can break the above equation down into finding the probability on a per measurement basis:

$$
P(X|w) = \prod_{i}^{N} p(x_i|w)
$$

So, if we multiply the probability that we would see each individual data point - given our weight guess - then we can find one number comparing our weight guess to all of our data.

The peak in the likelihood is the weight of the apple.

To make it computationally easier,

$$
\log P(X|w) = \log \prod_{i}^{N} p(x_i|w) = \sum_{i}^{N} \log p(d_i|w)
$$



a) Why did we use log likelihood? Is it ok to do so?

b) do the grid approximation and complete the cell below

In [3]:
from scipy.stats import norm
import numpy as np

weight_grid = np.linspace(0, 100)
print(weight_grid)

            ######################################################
        ###################### TO DO #########################
        ######################################################
        #                                                    #
        #                                                    #
        ######################################################
        ######################################################
        ######################################################

"""
# Calculate the maximum likelihood estimate of a parameter in a normal distribution.
# First calculate the log likelihoods for a range of weight guesses.
# For each weight guess, assume that the data comes from a normal distribution with that mean and a standard deviation of 10.
# Then calculate the log of the probability density function (pdf) of the data under this assumption.
# The sum of these log pdf values is the total log likelihood for that weight guess.
# After calculating the log likelihoods for all weight guesses, find the weight guess with the maximum log likelihood.
# This is the maximum likelihood estimate of the weight.
"""

[  0.           2.04081633   4.08163265   6.12244898   8.16326531
  10.20408163  12.24489796  14.28571429  16.32653061  18.36734694
  20.40816327  22.44897959  24.48979592  26.53061224  28.57142857
  30.6122449   32.65306122  34.69387755  36.73469388  38.7755102
  40.81632653  42.85714286  44.89795918  46.93877551  48.97959184
  51.02040816  53.06122449  55.10204082  57.14285714  59.18367347
  61.2244898   63.26530612  65.30612245  67.34693878  69.3877551
  71.42857143  73.46938776  75.51020408  77.55102041  79.59183673
  81.63265306  83.67346939  85.71428571  87.75510204  89.79591837
  91.83673469  93.87755102  95.91836735  97.95918367 100.        ]


'\n# Calculate the maximum likelihood estimate of a parameter in a normal distribution.\n# First calculate the log likelihoods for a range of weight guesses.\n# For each weight guess, assume that the data comes from a normal distribution with that mean and a standard deviation of 10.\n# Then calculate the log of the probability density function (pdf) of the data under this assumption.\n# The sum of these log pdf values is the total log likelihood for that weight guess.\n# After calculating the log likelihoods for all weight guesses, find the weight guess with the maximum log likelihood.\n# This is the maximum likelihood estimate of the weight.\n'

Play around with the code and try to answer the following questions regarding MLE and MAP. You can draw plots to visualize as well.

In [6]:
import numpy as np
from scipy.stats import norm, invgamma


# The barrel of apples
# The average apples is between 70-100 g
BARREL = np.random.normal(loc=85, scale=20, size=100)
# Grid
WEIGHT_GUESSES = np.linspace(1, 200, 100)
ERROR_GUESSES = np.linspace(.1, 50, 100)

# NOTE: Try changing the scale error
# in practice, you would not know this number
SCALE_ERR = 5

# NOTE: Try changing the number of measurements taken
N_MEASURMENTS = 10

# NOTE: Try changing the prior values and distributions
PRIOR_WEIGHT = norm(50, 1).logpdf(WEIGHT_GUESSES)
PRIOR_ERR = invgamma(4).logpdf(ERROR_GUESSES)

LOG_PRIOR_GRID = np.add.outer(PRIOR_ERR, PRIOR_WEIGHT)


def read_scale(apple):
    return apple + np.random.normal(loc=0, scale=SCALE_ERR)


def get_log_likelihood_grid(measurments):
    log_liklelihood = [
        [
            norm(weight_guess, error_guess).logpdf(measurments).sum()
            for weight_guess in WEIGHT_GUESSES
        ]
        for error_guess in ERROR_GUESSES
    ]
    return np.asarray(log_liklelihood)


def get_mle(measurements):
    """
    Calculate the log-likelihood for each measurement in the grid.
    Find the index of the maximum log-likelihood in the grid.
    Return the weight guess corresponding to the maximum log-likelihood.
    """
    ll_grid = get_log_likelihood_grid(measurements)

    sigma_hat_index, mu_hat_index = np.unravel_index(np.argmax(ll_grid), ll_grid.shape)
    # print(norm(WEIGHT_GUESSES[mu_hat_index], ERROR_GUESSES[sigma_hat_index]).logpdf(measurments).sum())
    # print(ll_grid[(sigma_hat_index, mu_hat_index)])
    return WEIGHT_GUESSES[mu_hat_index]


def get_map(measurements):
    """
    Calculate the log-likelihood for each measurement in the grid.
    Add the log prior to the log likelihood to get the log posterior.
    Find the index of the maximum log posterior in the grid.
    Return the weight guess corresponding to the maximum log posterior.
    """
    ll_grid = get_log_likelihood_grid(measurements)
    sigma_hat_index, mu_hat_index = np.unravel_index(np.argmax(ll_grid+LOG_PRIOR_GRID.T), ll_grid.shape)

    return WEIGHT_GUESSES[mu_hat_index]


# Pick an apple at random
apple = np.random.choice(BARREL)

# weight the apple
measurments = np.asarray([read_scale(apple) for _ in range(N_MEASURMENTS)])
# ll_grid = get_log_likelihood_grid(measurments)
# print(ll_grid)
# print(np.max(ll_grid))
# print(ll_grid.shape)

print(f"Average measurement: {measurments.mean():.3f} g")
print(f"Maximum Likelihood estimate: {get_mle(measurments):.3f} g")
print(f"Maximum A Posterior estimate: {get_map(measurments):.3f} g")
print(f"The true weight of the apple was: {apple:.3f} g")

Average measurement: 114.854 g
Maximum Likelihood estimate: 115.576 g
Maximum A Posterior estimate: 113.566 g
The true weight of the apple was: 114.398 g


<h3><i><i> Questions</h3>
1.
How sensitive is the MAP measurement to the choice of prior?

<h3><i><i></h3>
2. How sensitive is the MLE and MAP answer to the grid size?
